## 환경설정

In [1]:
!pip install -qU ragas langchain langchain-community langchain-openai langchain-chroma langchain-text-splitters pypdf rapidfuzz langgraph gdown chromadb langchain-huggingface sentence-transformers -q

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_F1")

In [3]:
# 필요한 라이브러리 임포트
import os
import pandas as pd
from pprint import pprint

In [4]:
# LangChain 관련 라이브러리
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [5]:
# RAGAS 관련 라이브러리
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.persona import Persona
from ragas.testset import TestsetGenerator
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

## 01.합성데이터 생성

In [6]:
!pip install gdown ragas langchain-openai -q

In [7]:
!pip install -U langchain-chroma -q

### 벡터DB 다운

In [8]:
import os
import shutil
import tempfile
import gdown
from pathlib import Path

# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 구글 드라이브 폴더를 다운로드하는 함수
def download_drive_folder_to_chroma_db(folder_url: str, target_dir: Path):
    target_dir = Path(target_dir).resolve()
    if target_dir.exists():
        shutil.rmtree(target_dir)
    target_dir.mkdir(parents=True, exist_ok=True)

    with tempfile.TemporaryDirectory() as td:
        gdown.download_folder(url=folder_url, output=td, quiet=False, use_cookies=False)
        entries = [Path(td) / name for name in os.listdir(td)]
        src_root = entries[0] if len(entries) == 1 and entries[0].is_dir() else Path(td)

        for p in src_root.iterdir():
            dst = target_dir / p.name
            if dst.exists():
                shutil.rmtree(dst) if dst.is_dir() else dst.unlink()
            shutil.move(str(p), str(dst))

    if not (target_dir / "chroma.sqlite3").exists():
        raise RuntimeError(f"'chroma.sqlite3' 파일이 다운로드되지 않았습니다: {target_dir}")

# 벡터 DB 다운로드 실행
FOLDER_URL = "https://drive.google.com/drive/u/0/folders/1YfEr9ZmnNR4Y9ekBR-ucfW3fdea50l9H"
DB_PATH = Path("./chroma_google_api_db")
download_drive_folder_to_chroma_db(FOLDER_URL, DB_PATH)

print(f"벡터 DB 다운로드 완료. 경로: {DB_PATH}")

Retrieving folder contents


Retrieving folder 11pYjOTHS1iKVQxTeGoj2a55QQjOH78oX 72417a29-586e-4f2f-b51b-02e9eaf7b268
Processing file 1kGB3icj_su9i7Z5nQOn0E40MZeHMdD73 data_level0.bin
Processing file 11nnNEdff_opCv8NeJ-FK_plSpfrmQg4l header.bin
Processing file 1SzVx1uY54eaADwngba89WUHMAeBCUYqc index_metadata.pickle
Processing file 1t_X4cSOKNznIZxfEuR7gj3-3VedWacEF length.bin
Processing file 1ZExbLv6ihyiLrrTSYbBJavdEmX19BOpv link_lists.bin
Processing file 1s36YP5HpMRdclvt7ToOtqPuAKcuOCR7- chroma.sqlite3


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1kGB3icj_su9i7Z5nQOn0E40MZeHMdD73
To: /tmp/tmpypwh5a94/72417a29-586e-4f2f-b51b-02e9eaf7b268/data_level0.bin
100%|██████████| 84.8M/84.8M [00:01<00:00, 66.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=11nnNEdff_opCv8NeJ-FK_plSpfrmQg4l
To: /tmp/tmpypwh5a94/72417a29-586e-4f2f-b51b-02e9eaf7b268/header.bin
100%|██████████| 100/100 [00:00<00:00, 305kB/s]
Downloading...
From: https://drive.google.com/uc?id=1SzVx1uY54eaADwngba89WUHMAeBCUYqc
To: /tmp/tmpypwh5a94/72417a29-586e-4f2f-b51b-02e9eaf7b268/index_metadata.pickle
100%|██████████| 1.84M/1.84M [00:00<00:00, 20.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1t_X4cSOKNznIZxfEuR7gj3-3VedWacEF
To: /tmp/tmpypwh5a94/72417a29-586e-4f2f-b51b-02e9eaf7b268/length.bin
100%|██████████| 80.1k/80.1k [00:00<00:00, 10.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=

벡터 DB 다운로드 완료. 경로: chroma_google_api_db



Download completed


In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# 임베딩 모델 초기화 // HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    encode_kwargs={"normalize_embeddings": True},
)

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name="langchain", ##### 원문: collection_name="langchain"
    persist_directory=str(DB_PATH),
)

print(f"벡터 DB 로드 완료. 저장된 Document 개수: {len(vector_store.get()['ids'])}")

벡터 DB 로드 완료. 저장된 Document 개수: 20017


In [10]:
# 벡터 DB 내용 확인
db_contents = vector_store.get(
    include=['metadatas', 'documents']
)

print("### 벡터 DB의 첫 3개 문서 내용 ###")
for i in range(3):
    print("--- 문서", i+1, "---")
    print("내용:", db_contents['documents'][i])
    print("메타데이터:", db_contents['metadatas'][i])
    print("-" * 20)

### 벡터 DB의 첫 3개 문서 내용 ###
--- 문서 1 ---
내용: Source URL: https://developers.google.com/workspace/drive/api/reference/rest/v3/User?hl=ko

이 페이지는 Cloud Translation API [https://cloud.google.com/translate/?hl=ko]를 통해 번역되었습니다.
Switch to English
홈 [https://developers.google.com/?hl=ko]
Google Workspace [https://developers.google.com/workspace?hl=ko]
Google Drive [https://developers.google.com/workspace/drive?hl=ko]
참조 [https://developers.google.com/workspace/drive/api/reference/rest/v3?hl=ko]
의견 보내기
User
bookmark_border
Drive 사용자에 관한 정보입니다.
JSON 표현
{
  "displayName": string,
  "kind": string,
  "me": boolean,
  "permissionId": string,
  "emailAddress": string,
  "photoLink": string
}
필드
displayName
string
출력 전용입니다. 이 사용자의 표시 가능한 일반 텍스트 이름입니다.
kind
string
출력 전용입니다. 리소스 종류를 식별합니다. 값: 고정 문자열 drive#user.
me
boolean
출력 전용입니다. 이 사용자가 요청하는 사용자인지 여부입니다.
permissionId
string
출력 전용입니다. 권한 리소스에 표시되는 사용자의 ID입니다.
emailAddress
string
출력 전용입니다. 사용자의 이메일 주소입니다. 사용자가 요청자에게 이메일 주소를 표시하지 않은 경우 특정 컨텍스트에 표시되지 않을 수

### 합성 데이터셋 생성

RAGAS를 사용하여 다양한 페르소나 기반의 질문-답변 데이터셋을 자동 생성한다.

In [11]:
# 데이터셋을 DataFrame으로 변환
import pandas as pd

file_path = './ragas_dataset.csv'
dataset_df = pd.read_csv(file_path)
pd.set_option('display.max_colwidth', None)

dataset_df

,user_input,reference_contexts,reference,synthesizer_name
0,Firestore에서 백업 정보를 조회하려면 어떤 권한이 필요한가요?,['Q: Firestore에서 백업 정보를 조회하기 위해 필요한 권한은 무엇인가요?\nA: Firestore에서 백업 정보를 조회하기 위해 필요한 권한은 다음과 같습니다: \n- datastore.backups.get \n- datastore.backups.list'],Firestore에서 백업 정보를 조회하기 위해서는 datastore.backups.get과 datastore.backups.list 권한이 필요합니다.,single_hop_specific_query_synthesizer
1,PHP Firestore add() 메서드 예시에서 Japan 값은 어떻게 사용되나요?,"[""Q: PHP에서 Firestore에 문서를 추가할 때 사용하는 add() 메서드의 예시는 무엇인가요?\nA: PHP에서 Firestore에 문서를 추가할 때 사용하는 add() 메서드의 예시는 다음과 같습니다.\n\n```php\n$data = [\n\t'name' => 'Tokyo',\n\t'country' => 'Japan'\n];\n$addedDocRef = $db->collection('samples/php/cities')->add($data);\nprintf('Added document with ID: %s' . PHP_EOL, $addedDocRef->id());\n```""]",PHP Firestore add() 메서드 예시에서 Japan 값은 country 필드의 값으로 사용됩니다.,single_hop_specific_query_synthesizer
2,type 필드 언제 생략대요?,"[""Q: StandardSqlField에서 'type' 필드가 생략될 수 있는 경우는 어떤 경우인가요?\nA: 'type' 필드는 명시적으로 지정되지 않은 경우에 생략될 수 있습니다. 예를 들어, CREATE FUNCTION 문에서 반환 유형을 생략할 수 있습니다.""]",'type' 필드는 명시적으로 지정되지 않은 경우에 생략될 수 있습니다.,single_hop_specific_query_synthesizer
3,BUCKET_NAME 뭐에요?,"['Q: ExportDocuments 메서드를 호출할 때, 어떤 형식의 URI를 출력 URI로 지정해야 하나요?\nA: ExportDocuments 메서드를 호출할 때, 출력 URI(output_uri_prefix)는 Google Cloud Storage URI 형식이어야 하며, 다음과 같은 형식을 따라야 합니다: gs://BUCKET_NAME[/NAMESPACE_PATH]. 여기서 BUCKET_NAME은 Google Cloud Storage 버킷의 이름이고, NAMESPACE_PATH는 선택적 Google Cloud Storage 네임스페이스 경로입니다.']",BUCKET_NAME은 Google Cloud Storage 버킷의 이름입니다.,single_hop_specific_query_synthesizer
4,Google OAuth 2.0 서버에 요청을 전송할 때 어떤 엔드포인트를 사용해야 하나요?,"['Q: Google의 OAuth 2.0 서버에 요청을 전송하는 방법은 무엇인가요?\nA: 사용자 승인을 받으려면 https://accounts.google.com/o/oauth2/v2/auth에서 Google 승인 서버에 요청을 전송하세요. 이 엔드포인트는 활성 세션 조회를 처리하고, 사용자를 인증하고, 사용자 동의를 획득합니다. 엔드포인트는 SSL을 통해서만 액세스할 수 있으며 HTTP (비 SSL) 연결은 거부합니다.']",Google OAuth 2.0 서버에 요청을 전송하려면 https://accounts.google.com/o/oauth2/v2/auth 엔드포인트를 사용해야 합니다.,single_hop_specific_query_synthesizer
5,peopleService 쓰면 연락처 삭제 어떻게 해요?,"['Q: 기존 연락처를 삭제하기 위한 Python 코드 예시는 무엇인가요?\nA: 기존 연락처를 삭제하기 위한 Python 코드는 다음과 같습니다.\n```python\npeopleService.people().deleteContact(""resource_name"").execute()\n```']","peopleService.people().deleteContact(""resource_name"").execute() 코드를 사용하면 기존 연락처를 삭제할 수 있습니다.",single_hop_specific_query_synthesizer
6,"OPEN API를 처음 사용하는 사람으로서, projects.databases.collectionGroups.fields.get 메서드의 성공적인 응답에 포함된 Field 인스턴스에 대해 더 자세히 알고 싶습니다. Field 문서가 무엇이며, 이 문서를 참고하면 어떤 정보를 얻을 수 있는지 설명해 주실 수 있나요?",['Q: projects.databases.collectionGroups.fields.get 메서드의 성공적인 응답은 어떤 형식인가요?\nA: 성공적인 응답 본문은 Field 인스턴스를 포함합니다. Field에 대한 자세한 내용은 [Field 문서](https://cloud.google.com/firestore/docs/reference/rest/v1/projects.databases.collectionGroups.fields#Field)를 참조하세요.'],projects.databases.collectionGroups.fields.get 메서드의 성공적인 응답 본문에는 Field 인스턴스가 포함되어 있습니다. Field에 대한 자세한 내용은 Field 문서를 참조하실 수 있습니다. Field 문서는 Field 인스턴스에 대한 정보를 제공하는 자료입니다.,single_hop_specific_query_synthesizer
7,캘린더에 이벤드 공유하려면 어떤 필드 써야되나요? 캘린더에서 공유 속성 추가하는 방법 자세히 알려주세요.,['Q: 이벤트에 공유 속성을 추가하려면 어떤 필드를 사용해야 하나요?\nA: 이벤트에 공유 속성을 추가하려면 `extendedProperties.shared` 필드를 사용해야 합니다. 이 필드는 다른 참석자의 캘린더에 있는 일정 사본 간에 공유되는 속성을 포함하는 객체입니다.'],이벤트에 공유 속성을 추가하려면 `extendedProperties.shared` 필드를 사용해야 합니다. 이 필드는 다른 참석자의 캘린더에 있는 일정 사본 간에 공유되는 속성을 포함하는 객체입니다.,single_hop_specific_query_synthesizer
8,usercreds 비활성화 하면 응답에 usercreds 정보가 어떻게 나오는지 자세히 알려주세여 usercreds가 뭔지도 잘 모르겠어요,['Q: userCreds 비활성화 요청의 성공적인 응답은 어떤 형태인가요?\nA: userCreds 비활성화 요청이 성공하면 응답 본문에는 `UserCreds` 인스턴스가 포함됩니다. 이 인스턴스는 비활성화된 userCreds에 대한 정보를 담고 있습니다.'],userCreds 비활성화 요청이 성공하면 응답 본문에는 UserCreds 인스턴스가 포함됩니다. 이 인스턴스는 비활성화된 userCreds에 대한 정보를 담고 있습니다.,single_hop_specific_query_synthesizer
9,"나 Search Ads 360 API 쓰고 싶은데 광고 데이터 보려면 무슨 OAuth 2.0 범위 써야 되는지 잘 모르겠어요, 알려줄 수 있나요?",['Q: Search Ads 360 API에서 광고 데이터를 보기 위해 필요한 OAuth 2.0 범위는 무엇인가요?\nA: Search Ads 360 API에서 광고 데이터를 보기 위해 필요한 OAuth 2.0 범위는 https://www.googleapis.com/auth/doubleclicksearch입니다.'],Search Ads 360 API에서 광고 데이터를 보기 위해 필요한 OAuth 2.0 범위는 http

## 02.RAG 체인 구성

검색기와 생성 모델을 결합한 RAG 체인을 구성한다.

### 우리 랭그래프

In [12]:
from typing import TypedDict, List, Dict, Any, Optional
from pathlib import Path
import os
import shutil
import tempfile
import gdown
import json
import torch

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

### retriever.py
- -> 코랩 기준으로 파일 저장 위치 수정
- embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    encode_kwargs={"normalize_embeddings": True},

In [13]:
HERE = "."
DB_DIR = os.path.join(HERE, "chroma_google_api_db")
COLLECTION_NAME = "langchain"
EMBED_MODEL = "BAAI/bge-m3"
TOP_K = 5

embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    encode_kwargs={"normalize_embeddings": True},
)

def retriever_setting():
    if not os.path.isdir(DB_DIR):
        print("ChromaDB 디렉토리가 없습니다. 구글 드라이브에서 다운로드합니다.")
        return None

    print(f"벡터 DB '{DB_DIR}' 로드 중...")
    vs = Chroma(
        collection_name=COLLECTION_NAME,
        persist_directory=DB_DIR,
        embedding_function=embeddings,
    )
    retriever = vs.as_retriever(search_kwargs={"k": TOP_K})
    print("검색기 설정 완료")
    return retriever

### rag2.py & rag.py
- basic_chain_setting... 어떤 파일 함수를 써야 하는지?

In [14]:
def basic_chain_setting(): # rag2.py
    llm = ChatOpenAI(model="gpt-4o", temperature=0)

    basic_prompt = PromptTemplate.from_template(
        """
    당신은 api 문서 관련 전문 챗봇으로서 사용자의 질문에 정확하고 친절하게 답변해야 합니다.
    아래 제공되는 문서에 없는 내용은 절대 답변에 포함하지 말고, 문서 내에서만 답변 내용을 찾아서 제공하세요.
    만약 사용자 질문이 구글 api 문서에 대한 질문이 아니라면, 아래 문서는 무시하고 일상 질문에 대해서만 답변하세요.

    문서 : {context}

    이전 대화 내역 : {history}

    이번 사용자 질문 : {question}
    """
    )

    basic_chain = basic_prompt | llm | StrOutputParser()

    return basic_chain


def query_setting():
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        model_kwargs={"response_format": {"type": "json_object"}},
    )

    query_prompt = PromptTemplate.from_template(
        """
       유저의 채팅 히스토리와 현재 질문이 주어집니다.


       **중요**: 이전 대화 맥락을 반드시 고려해서 질문을 생성하세요.
       - 현재 질문이 이전 대화와 연관되어 있다면, 이전 맥락을 포함한 통합된 질문을 만들어주세요.

       - 예: 바로 전에 "People API 연락처 조회"에 대해 이야기하고 나서, "그럼 프로필 수정은?"이라는 질문이 나오면 "People API에서 프로필 수정 방법"으로 통합해주세요.
       - 주의사항: 이전에 "People API 연락첯 조회"에 대해 이야기하고 나서, "Firebase"와 같이 다른 api에 대한 대화 내용이 나온 후 "프로필 수정은?"이라는 질문이 나오면 마지막 대화 맥락에 맞춰서, "Firebase에서 프로필 수정 방법"과 같이 통합해야 합니다.

       - 이전 대화에서 이미 답변이 나온 질문은 생성하지 마세요.
       - 질문은 1개가 될 수도 있고 여러개가 될 수도 있습니다.

       대화 히스토리: {rewritten}

       JSON 반환 형태:
       {{"questions": ["맥락을 고려한 통합 질문 1", "맥락을 고려한 통합 질문 2", ...]}}
       """
    )

    def parse_json(response):
        return json.loads(response.content)  # response.content 사용

    chain = query_prompt | llm | parse_json
    return chain

### langgraph_node.py

In [15]:
basic_chain = basic_chain_setting()
retriever = retriever_setting()
query_chain = query_setting()


class ChatState(TypedDict, total=False):
    question: str  # 유저 질문
    answer: str  # 모델 답변
    rewritten: str  # 통합된 질문
    queries: List[str]  # 쿼리(질문들)
    search_results: List[str]  # 벡터 DB 검색 결과들
    messages: List[Dict[str, str]]  # 사용자 및 모델의 대화 히스토리


# (1) 사용자 질문 + 히스토리 통합 → 통합된 질문과 쿼리 추출
def extract_queries(state: ChatState) -> ChatState:
    user_text = state["question"]

    # 히스토리에서 최근 몇 개의 메시지를 가져와서 통합 질문을 생성
    messages = state.get("messages", [])

    # 최근 4개 메시지만 사용
    history_tail = messages[-4:] if messages else []
    context = history_tail.copy()

    # 현재 사용자 질문 추가
    context.append({"role": "user", "content": user_text})
    state["rewritten"] = context

    return state


# (2) LLM에게 질문 분리를 시킨다
def split_queries(state: ChatState) -> ChatState:
    rewritten = state.get("rewritten")

    response = query_chain.invoke({"rewritten": rewritten})
    state["queries"] = response["questions"]  # questions 리스트만 저장

    return state


# (3) 벡터 DB 툴 호출
def search_tool(query: str):
    """질문을 바탕으로 벡터 DB에서 결과 검색"""
    return retriever.invoke(query)  # retriever는 DB 검색 로직을 호출


# (4) 기본 답변 생성 노드
def langgraph_node(state: ChatState) -> Dict[str, Any]:
    history = state.get("messages", [])
    """질문에 대한 기본 답변 생성"""
    queries = state["queries"]
    print(f"생성된 질문 리스트 {queries}")
    search_results = []

    # 각 쿼리마다 벡터 DB 검색
    for query in queries:
        print(f"{query} 검색중...")
        results = search_tool(query)
        search_results.append(results)  # 검색된 결과들을 모아서 저장

    # 검색된 결과를 바탕으로 답변 생성
    answer = basic_chain.invoke(
        {
            "question": state["question"],
            "history": history,
            "context": "\n".join([str(res) for res in search_results]),
        }
    ).strip()

    state["search_results"] = search_results
    state["answer"] = answer

    return state  # 답변을 반환

벡터 DB './chroma_google_api_db' 로드 중...
검색기 설정 완료


/tmp/ipython-input-3532891692.py:18: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs = Chroma(


### langgraph_setting_edit.py
- langgraph_setting.py 이건 쓰이는지?

In [16]:
# 그래프 설정
def graph_setting_edit():
    # LangGraph 정의
    graph = StateGraph(ChatState)

    # 노드 등록
    graph.add_node("extract_queries", extract_queries)  # 질문 통합 + 쿼리 추출 노드
    graph.add_node("split_queries", split_queries)  # 질문 분리 툴
    graph.add_node("basic", langgraph_node)  # 기본 답변 노드

    # 시작 노드 정의
    graph.set_entry_point("extract_queries")

    # 흐름 설정
    graph.add_edge("extract_queries", "split_queries")  # 질문 추출 후 분리
    graph.add_edge("split_queries", "basic")  # 쿼리 분리 후 기본 답변 노드로 넘어감
    graph.add_edge("basic", END)  # 기본 답변 후 종료

    # 그래프 컴파일
    memory = MemorySaver()
    compiled_graph = graph.compile(checkpointer=memory)

    return compiled_graph

### 실행
- main2.py

In [17]:
# RAGAS 평가 루프
# : 답변(answer)뿐만 아니라 검색된 문서들(retrieved_contexts)도 함께 반환해야 함

graph = graph_setting_edit()


def run_langraph(user_input, config_id, chat_history=None):
    config = {"configurable": {"thread_id": config_id}}

    if chat_history is None:
        chat_history = []

    result = graph.invoke(
        {"messages": chat_history, "question": user_input}, config=config
    )

    return {
        "answer": result["answer"],
        "retrieved_contexts": result["search_results"]
    }

## 03.RAGAS기반 평가
### RAGAS 주요 평가 지표
- **Faithfulness(충실도)**: 생성된 답변이 주어진 컨텍스트 정보에 얼마나 충실한지를 평가합니다. 답변 내용이 컨텍스트에서 실제로 뒷받침되는지 보는 지표입니다.
- **Answer Relevancy(답변 관련성)**: 답변이 원 질문과 얼마나 관련성이 높은지를 측정합니다.
- **Context Precision(컨텍스트 정밀도)**: 검색된 컨텍스트 문서가 질문에 적절한 정보인지, 관련된 문서가 상위에 있는지를 평가합니다.
- **Context Recall(컨텍스트 재현율)**: 답변을 생성하는 데 필요한 컨텍스트를 얼마나 잘 검색했는지 평가합니다.

In [18]:
# 평가용 데이터 로드
eval_dataset = dataset_df[['user_input', 'reference_contexts', 'reference']]
eval_dataset.head()

,user_input,reference_contexts,reference
0,Firestore에서 백업 정보를 조회하려면 어떤 권한이 필요한가요?,['Q: Firestore에서 백업 정보를 조회하기 위해 필요한 권한은 무엇인가요?\nA: Firestore에서 백업 정보를 조회하기 위해 필요한 권한은 다음과 같습니다: \n- datastore.backups.get \n- datastore.backups.list'],Firestore에서 백업 정보를 조회하기 위해서는 datastore.backups.get과 datastore.backups.list 권한이 필요합니다.
1,PHP Firestore add() 메서드 예시에서 Japan 값은 어떻게 사용되나요?,"[""Q: PHP에서 Firestore에 문서를 추가할 때 사용하는 add() 메서드의 예시는 무엇인가요?\nA: PHP에서 Firestore에 문서를 추가할 때 사용하는 add() 메서드의 예시는 다음과 같습니다.\n\n```php\n$data = [\n\t'name' => 'Tokyo',\n\t'country' => 'Japan'\n];\n$addedDocRef = $db->collection('samples/php/cities')->add($data);\nprintf('Added document with ID: %s' . PHP_EOL, $addedDocRef->id());\n```""]",PHP Firestore add() 메서드 예시에서 Japan 값은 country 필드의 값으로 사용됩니다.
2,type 필드 언제 생략대요?,"[""Q: StandardSqlField에서 'type' 필드가 생략될 수 있는 경우는 어떤 경우인가요?\nA: 'type' 필드는 명시적으로 지정되지 않은 경우에 생략될 수 있습니다. 예를 들어, CREATE FUNCTION 문에서 반환 유형을 생략할 수 있습니다.""]",'type' 필드는 명시적으로 지정되지 않은 경우에 생략될 수 있습니다.
3,BUCKET_NAME 뭐에요?,"['Q: ExportDocuments 메서드를 호출할 때, 어떤 형식의 URI를 출력 URI로 지정해야 하나요?\nA: ExportDocuments 메서드를 호출할 때, 출력 URI(output_uri_prefix)는 Google Cloud Storage URI 형식이어야 하며, 다음과 같은 형식을 따라야 합니다: gs://BUCKET_NAME[/NAMESPACE_PATH]. 여기서 BUCKET_NAME은 Google Cloud Storage 버킷의 이름이고, NAMESPACE_PATH는 선택적 Google Cloud Storage 네임스페이스 경로입니다.']",BUCKET_NAME은 Google Cloud Storage 버킷의 이름입니다.
4,Google OAuth 2.0 서버에 요청을 전송할 때 어떤 엔드포인트를 사용해야 하나요?,"['Q: Google의 OAuth 2.0 서버에 요청을 전송하는 방법은 무엇인가요?\nA: 사용자 승인을 받으려면 https://accounts.google.com/o/oauth2/v2/auth에서 Google 승인 서버에 요청을 전송하세요. 이 엔드포인트는 활성 세션 조회를 처리하고, 사용자를 인증하고, 사용자 동의를 획득합니다. 엔드포인트는 SSL을 통해서만 액세스할 수 있으며 HTTP (비 SSL) 연결은 거부합니다.']",Google OAuth 2.0 서버에 요청을 전송하려면 https://accounts.google.com/o/oauth2/v2/auth 엔드포인트를 사용해야 합니다.


- user_input: ragas가 생성한 합성 데이터셋(dataset_df)에서 가져온 질문

- retrieved_contexts: 랭그래프 RAG 체인(run_langraph)이 질문을 바탕으로 벡터 DB에서 검색해 온 문서들

- response: 랭그래프 RAG 체인이 검색된 문서를 근거로 최종적으로 생성한 답변

- reference: ragas가 생성한 합성 데이터셋(dataset_df)에 들어있던 '정답(ground truth)' -> RAG 체인이 만든 답변을 평가하는 기준이 됨

In [19]:
import pandas as pd
from datasets import Dataset
import uuid

evaluated_dataset = []

for _, row in dataset_df.iterrows():
    query = row.user_input  # 사용자 입력 - 합성 데이터셋에서 가져옴

    # run_langraph 함수 호출
    rag_result = run_langraph(query, config_id=str(uuid.uuid4()), chat_history=None)

    evaluated_dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [doc.page_content for docs_list in rag_result["retrieved_contexts"] for doc in docs_list],
            "response": rag_result["answer"],
            "reference": row['reference']
        }
    )

print("평가 데이터셋 준비 완료")

# RAGAS 평가 데이터셋 생성
ragas_evaluated_dataset = Dataset.from_list(evaluated_dataset)

생성된 질문 리스트 ['Firestore에서 백업 정보를 조회하기 위한 권한은 무엇인가요?']
Firestore에서 백업 정보를 조회하기 위한 권한은 무엇인가요? 검색중...
생성된 질문 리스트 ['PHP Firestore add() 메서드에서 Japan 값을 사용하는 방법은 무엇인가요?']
PHP Firestore add() 메서드에서 Japan 값을 사용하는 방법은 무엇인가요? 검색중...
생성된 질문 리스트 ['type 필드를 생략할 수 있는 경우는 어떤 상황인가요?']
type 필드를 생략할 수 있는 경우는 어떤 상황인가요? 검색중...
생성된 질문 리스트 ['BUCKET_NAME의 정의는 무엇인가요?']
BUCKET_NAME의 정의는 무엇인가요? 검색중...
생성된 질문 리스트 ['Google OAuth 2.0 서버에 요청을 전송할 때 사용하는 엔드포인트는 무엇인가요?']
Google OAuth 2.0 서버에 요청을 전송할 때 사용하는 엔드포인트는 무엇인가요? 검색중...
생성된 질문 리스트 ['People API에서 연락처 삭제 방법은 무엇인가요?']
People API에서 연락처 삭제 방법은 무엇인가요? 검색중...
생성된 질문 리스트 ['projects.databases.collectionGroups.fields.get 메서드의 성공적인 응답에 포함된 Field 인스턴스의 문서에서 어떤 정보를 얻을 수 있는지 설명해 주실 수 있나요?']
projects.databases.collectionGroups.fields.get 메서드의 성공적인 응답에 포함된 Field 인스턴스의 문서에서 어떤 정보를 얻을 수 있는지 설명해 주실 수 있나요? 검색중...
생성된 질문 리스트 ['캘린더에서 이벤트 공유를 위한 필드와 공유 속성 추가 방법은 무엇인가요?']
캘린더에서 이벤트 공유를 위한 필드와 공유 속성 추가 방법은 무엇인가요? 검색중...
생성된 질문 리스트 ['usercreds가 무엇인지 설명해주시고, 비활성화했을 때 응답에 usercreds 정보가 어떻

In [20]:
# csv 데이터 저장
ragas_evaluated_dataset.to_pandas().to_csv('ragas_evaluated_dataset_original.csv', index=False)

In [21]:
ragas_evaluated_dataset.to_pandas()

user_input  \
0                                                                                                                                                      Firestore에서 백업 정보를 조회하려면 어떤 권한이 필요한가요?   
1                                                                                                                                            PHP Firestore add() 메서드 예시에서 Japan 값은 어떻게 사용되나요?   
2                                                                                                                                                                            type 필드 언제 생략대요?   
3                                                                                                                                                                            BUCKET_NAME 뭐에요?   
4                                                                                                                                          Google OAuth 2.0 서버에 요청을 전송할 때 어떤 엔드포인트를 사용해야 하나요?   
5                                                                                                                                                             peopleService 쓰면 연락처 삭제 어떻게 해요?   
6            OPEN API를 처음 사용하는 사람으로서, projects.databases.collectionGroups.fields.get 메서드의 성공적인 응답에 포함된 Field 인스턴스에 대해 더 자세히 알고 싶습니다. Field 문서가 무엇이며, 이 문서를 참고하면 어떤 정보를 얻을 수 있는지 설명해 주실 수 있나요?   
7                                                                                                                                  캘린더에 이벤드 공유하려면 어떤 필드 써야되나요? 캘린더에서 공유 속성 추가하는 방법 자세히 알려주세요.   
8                                                                                                               usercreds 비활성화 하면 응답에 usercreds 정보가 어떻게 나오는지 자세히 알려주세여 usercreds가 뭔지도 잘 모르겠어요   
9                                                                                                           나 Search Ads 360 API 쓰고 싶은데 광고 데이터 보려면 무슨 OAuth 2.0 범위 써야 되는지 잘 모르겠어요, 알려줄 수 있나요?   
10    me want know how HTTP work for update calendar metadata, can you tell me what HTTP request look like and what calendarId mean, because I expert but want more detail, explain long way?   
11                                                                                                                                                 Google 프로젝트 소유자와 편집자가 관리해야 할 주요 사항은 무엇인가요?   
12                                                                                                                                                           컴퓨트 엔진 인스턴스에 서비스 어카운트 어떻게 연결하나요?   
13                                                                                                                                      users/jeff/tasks/my_task_id 문서의 done 필드 색인 크기는 얼마입니까?   
14                                                                                                                                                                     gcloud CLI 뭐에요 어떻게 써요?   
15                                                                                                      Unity에서 Firebase를 추가한 후 Android 빌드 오류를 방지하기 위해 Publishing Settings에서 어떤 설정을 조정해야 하나요?   
16                                                                                            httsp://firestore.googelapis.com/v1/{name=projects/*/databases/*}:exportDocuments 이거 어떻게 써야되나요?   
17                                                                               나 Firebase Authentication 쓰고 싶어요, 근데 Gradle 설정 어떻게 해야돼요? Firebase Android BoM 이거 왜 써야돼요, 뭐에요, 어떻게 쓰는지 알려주세요.   
18                                                                                                                     How use credentials when start Cloud Firestore async client in Python?   
19                                                          클라우드 파이어스토어에서 색인 빌드 타임이 어떻게 정해지는지 잘 모르겠는데, 혹시 클라우드 파이어스토어 색인 빌드 시간은 데이터가 많으면 더 오래 걸리는 건가요? 그리고 색인 빌드 시간에 영향을 주는 요소가 뭐가 있는지 궁금합니다.   
20                                                                                                                            

### RAGAS 평가 실행

**ragas.metrics 주요 클래스 설명**

- **LLMContextRecall**  
  
  주어진 답변이 원문(문맥)에서 제시된 정보를 얼마나 잘 회상(recall)했는지 평가하는 클래스이다.
  
  예를 들어, 원문에 중요한 사실 10개가 있는데 답변이 그중 몇 개를 잘 포함했는지 측정한다.
  즉, 잊지 않고 잘 회상했는지 보는 지표다.

- **Faithfulness**  
  답변이 원본(문맥) 정보에 기반하여 얼마나 충실하고 일관되게 생성되었는지 평가하는 클래스이다.

  답변 내 내용이 문맥과 어긋나지 않고, 문맥에서 유추 가능한 사실들로만 이루어졌는지를 본다.

  낮은 faithfulness 값은 모델이 문맥에 없는 정보를 만들어내거나 왜곡하여 답변함을 의미할 수 있어, 환각 발생 정도를 간접적으로 보여준다.

- **FactualCorrectness**  
  답변의 내용이 실제 사실과 얼마나 정확하게 일치하는지를 평가하는 클래스이다.

  문맥뿐 아니라 절대적인 사실관계(정확성)와 맞는지 따진다.

  즉, 문맥이 아니라 객관적인 사실에 맞는지를 측정하는 지표다.

In [22]:
# LLM 래퍼 생성
evaluator_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, request_timeout=120)
evaluator_llm = LangchainLLMWrapper(evaluator_llm)

# 평가 메트릭 정의
metrics = [
    LLMContextRecall(),    # 검색된 컨텍스트의 회수율
    Faithfulness(),        # 생성된 답변의 충실도
    FactualCorrectness()   # 사실적 정확성
]

In [23]:
# 평가 실행
import os

# 분당 토큰 한도 error -> 동시 호출 수를 4개로 제한
os.environ['RAGAS_CONCURRENCY'] = '4'

result = evaluate(
    dataset=ragas_evaluated_dataset,
    metrics=metrics,
    llm=evaluator_llm,
)

print(result)

Evaluating:   0%|          | 0/90 [00:00<?, ?it/s]

{'context_recall': 0.7500, 'faithfulness': 0.8157, 'factual_correctness(mode=f1)': 0.4373}


In [24]:
# 결과를 DataFrame으로 변환
result_df = result.to_pandas()

# 결과 저장
result_df.to_csv('ragas_evaluation_result_original.csv', index=False)

result_df.head()

user_input  \
0              Firestore에서 백업 정보를 조회하려면 어떤 권한이 필요한가요?   
1    PHP Firestore add() 메서드 예시에서 Japan 값은 어떻게 사용되나요?   
2                                    type 필드 언제 생략대요?   
3                                    BUCKET_NAME 뭐에요?   
4  Google OAuth 2.0 서버에 요청을 전송할 때 어떤 엔드포인트를 사용해야 하나요?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [25]:
result_df[['context_recall', 'faithfulness', 'factual_correctness(mode=f1)']].describe()

,context_recall,faithfulness,factual_correctness(mode=f1)
count,30.000000,30.000000,30.000000
mean,0.750000,0.815729,0.437333
std,0.416954,0.264270,0.248595
min,0.000000,0.000000,0.000000
25%,0.666667,0.700000,0.312500
50%,1.000000,0.966667,0.440000
75%,1.000000,1.000000,0.585000
max,1.000000,1.000000,0.910000
